In [ ]:
import numpy as np
import pandas as pd
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb
import time

In [ ]:
def load_data():
    sensor_data = pd.read_csv("../data/processed/causalDiscoveryData.csv", sep=";", names=[
        "location", "lat", "lon", "timestamp", "dayOfYear", "minuteOfDay", "dayOfWeek", "isWeekend", "pressure", "altitude", "pressure_sealevel", "temperature",
        "humidity", "p1", "p2", "p0", "durP1", "ratioP1", "durP2", "ratioP2",
        "apparent_temperature", "cloud_cover", "dew_point", "humidity", "ozone", "precip_intensity", "precip_probability",
        "precip_type", "pressure", "uv_index", "visibility", "wind_bearing", "wind_gust", "wind_speed"], true_values=["true"], false_values=["false"])

    sensor_data["timestamp"] = pd.to_datetime(sensor_data["timestamp"])
    sensor_data["isWeekend"] = sensor_data["isWeekend"].astype(int)

    sensor_data = sensor_data.sort_values(by=["location", "timestamp"])
    return sensor_data

In [ ]:
def extract_dataset(sensor_data):
    sensor_data = sensor_data[["location", "lat", "lon", "timestamp", "dayOfYear", "minuteOfDay", "dayOfWeek", "isWeekend", "temperature",
        "humidity", "p1", "p2", "apparent_temperature", "cloud_cover", "dew_point", "humidity", "precip_intensity", "precip_probability",
        "visibility", "wind_bearing", "wind_gust", "wind_speed"]]
    sensor716 = sensor_data[sensor_data["location"] == 716]
    sensor716.loc[sensor716["wind_gust"].isna(), "wind_gust"] = sensor716[sensor716["wind_gust"].isna()]["wind_speed"] 
    sensor716.loc[sensor716["wind_bearing"].isna(), "wind_bearing"] = 0.
    sensor716.loc[sensor716["cloud_cover"].isna(), "cloud_cover"] = 0.
    sensor716.loc[sensor716["precip_intensity"].isna(), "precip_intensity"] = 0.
    sensor716.loc[sensor716["precip_probability"].isna(), "precip_probability"] = 0.
    sensor716 = sensor716.dropna()
    return sensor716

In [ ]:
sensor716 = extract_dataset(sensor_data)

In [ ]:
def var_names():
    return ["dayOfYear", "minuteOfDay", "dayOfWeek", "isWeekend", "temperature",
        "humidity", "p1", "p2", "apparent_temperature", "cloud_cover", "dew_point", "humidity", "precip_intensity", "precip_probability",
        "visibility", "wind_bearing", "wind_gust", "wind_speed"]

def create_tigramite_dataframe(dataset):
    data = dataset[var_names()]
    datatime = dataset["timestamp"]

    dataframe = pp.DataFrame(data.values, datatime = datatime.values, var_names=var_names())
    return dataframe

In [ ]:
dataframe = create_tigramite_dataframe(extract_dataset(load_data()))

In [ ]:
start = time.time()
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(
    dataframe=dataframe,
    cond_ind_test=parcorr,
    verbosity=1)

correlations = pcmci.get_lagged_dependencies(tau_max=20)
end = time.time()
print(round(end - start, 2))

In [ ]:
lag_func_matrix = tp.plot_lagfuncs(val_matrix=correlations, setup_args={'var_names':var_names(), "figsize":(50,50), "x_base": 20
                                                                    })

In [ ]:
pcmci.verbosity = 1
results = pcmci.run_pcmci(tau_max=4, pc_alpha=None)

In [ ]:
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
pcmci.print_significant_links(
        p_matrix = results['p_matrix'], 
        q_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        alpha_level = 0.01)

In [ ]:
link_matrix = pcmci.return_significant_parents(pq_matrix=q_matrix,
                        val_matrix=results['val_matrix'], alpha_level=0.01)['link_matrix']

In [ ]:
tp.plot_graph(
    val_matrix=results['val_matrix'],
    link_matrix=link_matrix,
    var_names=var_names,
    link_colorbar_label='cross-MCI',
    node_colorbar_label='auto-MCI',
    )

In [ ]:
# Plot time series graph
tp.plot_time_series_graph(
    val_matrix=results['val_matrix'],
    link_matrix=link_matrix,
    var_names=var_names,
    link_colorbar_label='MCI',
    )

## Non-linear

In [ ]:
gpdc = GPDC(significance='analytic', gp_params=None)
gpdc.generate_and_save_nulldists(sample_sizes=range(495, 501),
    null_dist_filename='dc_nulldists.npz')
gpdc.null_dist_filename ='dc_nulldists.npz'
pcmci_gpdc = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=gpdc,
    verbosity=0)